### Examples Of Pyspark ML

#### INICIAMOS SESION

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dataframe').getOrCreate()

CARGAMOS CSV

training = spark.read.csv('data/test1.csv', header=True, inferSchema=True)

In [2]:
training = spark.read.csv('data/test1.csv', header=True, inferSchema=True)

In [3]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



En PySpark se trabaja de forma diferente. Tendremos que agrupar nuestras variables independientes de forma que queden todas en una columna y dentro de una lista, por lo que crearemos un vector de ensamblaje o "vector assembler", de tal modo que queden así esas variables independientes:
- [Age, Experience]

Lo que haremos con estas dos, será tratarlas como una nueva variable independiente:
- [Age, Experience] ----> nueva_variable_independiente

In [4]:
from pyspark.ml.feature import VectorAssembler 

In [5]:
feature_assembler = VectorAssembler(inputCols=['age', 'Experience'], outputCol='Independent features') # 1:46:27 del vídeo

In [6]:
output = feature_assembler.transform(training)

Veremos que se crea una nueva columna cuyos valores se corresponden a unos array con el contenido de aquellas variables independientes que hemos agrupado. Esto será nuestro input feature o lo que solíamos definir como train.

In [7]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



Seleccionamos las columnas que nos interesan para nuestro modelo: el train (Independent Features) y el test (Salary)

In [8]:
finalized_data = output.select('Independent features', 'Salary')
finalized_data.show()

+--------------------+------+
|Independent features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



A continuación, entrenaremos un modelo de regresión lineal.

In [9]:
from pyspark.ml.regression import LinearRegression

In [10]:
(train, test) = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independent features', labelCol='Salary')
regressor = regressor.fit(train)

In [11]:
regressor.coefficients

DenseVector([-258.8832, 1670.0508])

In [12]:
regressor.intercept

20543.147208121274

In [13]:
prediction = regressor.evaluate(test)

In [14]:
prediction.predictions.show()

+--------------------+------+------------------+
|Independent features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|16776.649746192812|
|          [24.0,3.0]| 20000|19340.101522842593|
+--------------------+------+------------------+



In [15]:
# Errores

prediction.meanAbsoluteError, prediction.meanSquaredError

(1218.2741116751095, 1795975.1604008244)

In [29]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()

In [30]:
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [31]:
iris['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [34]:
dt = pd.DataFrame(data=iris['data'], columns=iris['feature_names'])
dt['target'] = iris['target']

In [35]:
dt

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [39]:
dt.to_csv("iris.csv")
pyspark_iris = spark.read.csv('iris.csv', header=True, inferSchema=True)

In [40]:
pyspark_iris.show()

+---+-----------------+----------------+-----------------+----------------+------+
|_c0|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|
+---+-----------------+----------------+-----------------+----------------+------+
|  0|              5.1|             3.5|              1.4|             0.2|     0|
|  1|              4.9|             3.0|              1.4|             0.2|     0|
|  2|              4.7|             3.2|              1.3|             0.2|     0|
|  3|              4.6|             3.1|              1.5|             0.2|     0|
|  4|              5.0|             3.6|              1.4|             0.2|     0|
|  5|              5.4|             3.9|              1.7|             0.4|     0|
|  6|              4.6|             3.4|              1.4|             0.3|     0|
|  7|              5.0|             3.4|              1.5|             0.2|     0|
|  8|              4.4|             2.9|              1.4|             0.2|     0|
|  9